Сервис по продаже автомобилей с пробегом «Не бит, не крашен» разрабатывает приложение для привлечения новых клиентов. В нём можно быстро узнать рыночную стоимость своего автомобиля. В вашем распоряжении исторические данные: технические характеристики, комплектации и цены автомобилей. Вам нужно построить модель для определения стоимости. 

Заказчику важны:

- качество предсказания;
- скорость предсказания;
- время обучения.

# 1. Подготовка данных

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error as RMSE
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from catboost import CatBoostRegressor
import lightgbm as lgb
import time
import warnings
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score
from sklearn.metrics import make_scorer
from sklearn.model_selection import GridSearchCV

In [2]:
df = pd.read_csv('/datasets/autos.csv')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 16 columns):
DateCrawled          354369 non-null object
Price                354369 non-null int64
VehicleType          316879 non-null object
RegistrationYear     354369 non-null int64
Gearbox              334536 non-null object
Power                354369 non-null int64
Model                334664 non-null object
Kilometer            354369 non-null int64
RegistrationMonth    354369 non-null int64
FuelType             321474 non-null object
Brand                354369 non-null object
NotRepaired          283215 non-null object
DateCreated          354369 non-null object
NumberOfPictures     354369 non-null int64
PostalCode           354369 non-null int64
LastSeen             354369 non-null object
dtypes: int64(7), object(9)
memory usage: 43.3+ MB


In [4]:
df.head()

,DateCrawled,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Kilometer,RegistrationMonth,FuelType,Brand,NotRepaired,DateCreated,NumberOfPictures,PostalCode,LastSeen
0,2016-03-24 11:52:17,480,NaN,1993,manual,0,golf,150000,0,petrol,volkswagen,NaN,2016-03-24 00:00:00,0,70435,2016-04-07 03:16:57
1,2016-03-24 10:58:45,18300,coupe,2011,manual,190,NaN,125000,5,gasoline,audi,yes,2016-03-24 00:00:00,0,66954,2016-04-07 01:46:50
2,2016-03-14 12:52:21,9800,suv,2004,auto,163,grand,125000,8,gasoline,jeep,NaN,2016-03-14 00:00:00,0,90480,2016-04-05 12:47:46
3,2016-03-17 16:54:04,1500,small,2001,manual,75,golf,150000,6,petrol,volkswagen,no,2016-03-17 00:00:00,0,91074,2016-03-17 17:40:17
4,2016-03-31 17:25:20,3600,small,2008,manual,69,fabia,90000,7,gasoline,skoda,no,2016-03-31 00:00:00,0,60437,2016-04-06 10:17:21


##### Выборка очень большая - около 350.000 автомобилей. Больше всего пропусков наблюдается в признаке NotRepaired. Скорее всего, пропуск означает что машина не была в ремонте, по крайней мере это точно для большинства, а значит можно провести такую замену. Так же присутствует ощутимое количество пропусков в признаках VehicleType и FuelType, и немного меньше в model и Gearbox.
##### VehicleType(тип кузова) - очень важный параметр, и совершенно неясно как заполнить пропуски. Можно, например, взять наиболее популярный тип кузова среди данной модели, но это все равно чревато плохими данными. Учитывая размер выборки и количество пропусков, проще будет отказаться от этих данных. Думаю будет лучше так же поступить и с остальными данными, во избежание заведомо большого количества сомнительных вариантов в случае замены, и учитывая размер выборки даже после удаления пропусков

In [5]:
df['NotRepaired'] = df['NotRepaired'].fillna('no')

In [6]:
df = df.dropna()

In [7]:
df_category = df.copy()

In [8]:
df = pd.get_dummies(df, columns = ['VehicleType', 'Gearbox', 'Model', 'FuelType', 'Brand', 'NotRepaired'], drop_first = True)

In [9]:
df = df.drop_duplicates()

##### В выборке присутствуют данные с очень странными ценами. От автомобилей с ценой 0 точно надо избавиться, хотя цена в 100 евро тоже выглядит сомнительно

In [10]:
df[df['Price'] < 500].shape

(19283, 312)

In [11]:
df[df['Price'] < 100].shape

(5984, 312)

##### Удалим хотя бы данные где цена меньше 100, в основном это будут нулевые значения от которых точно никакого толку

In [12]:
df = df.query('Price > 99')

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 278137 entries, 2 to 354368
Columns: 312 entries, DateCrawled to NotRepaired_yes
dtypes: int64(7), object(3), uint8(302)
memory usage: 103.4+ MB


##### Согласен, потери велики, не будь выборка такой большой пришлось бы придумать что-нибудь получше. Зато в данном случае выборка все еще велика, и в ней точно нет неверных данных

##### Выделим признаки и целевой признак, и разделим на выборки, а так же отбросим признаки, не несущие особого смысла

In [14]:
features = df.drop(['Price', 'DateCrawled', 'DateCreated', 'PostalCode', 'LastSeen', 'NumberOfPictures'], axis = 1)
target = df['Price']

In [15]:
features_train, features_valid, target_train, target_valid = train_test_split(features, target,
                                                                              test_size = 0.33, random_state = 123)

In [16]:
scaler = StandardScaler()
numeric = ['RegistrationYear','Power','Kilometer']
scaler.fit(features_train.loc[:, numeric])
features_train.loc[:, numeric] = scaler.transform(features_train.loc[:, numeric])
features_valid.loc[:, numeric] = scaler.transform(features_valid.loc[:, numeric])
warnings.filterwarnings('ignore')

/opt/conda/lib/python3.7/site-packages/pandas/core/indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s
/opt/conda/lib/python3.7/site-packages/pandas/core/indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [17]:
category = ['VehicleType', 'Gearbox', 'Model', 'FuelType', 'Brand', 'NotRepaired']
#features_train.loc[:, category] = features_train.loc[:, category].astype('category')
#features_valid.loc[:, category] = features_valid.loc[:, category].astype('category')

In [18]:
category_features = df_category.drop(['Price', 'DateCrawled', 'DateCreated', 'PostalCode', 'LastSeen', 'NumberOfPictures'], axis = 1)
category_target = df_category['Price']

In [19]:
category_features[category] = category_features[category].astype('category')

In [20]:
category_features_train, category_features_test, category_target_train, category_target_test = train_test_split(
    category_features, category_target, test_size = 0.33, random_state = 123)

# 2. Обучение моделей

##### Сначала обучим несколько обычных моделей, а позже используем градиентный бустинг и сравним результаты

##### Начнем с линейной регрессии

In [21]:
%%time
start = time.time()
model_LR = LinearRegression()
model_LR.fit(features_train, target_train)
predictions_LR = model_LR.predict(features_valid)
score_LR = RMSE(target_valid, predictions_LR) ** 0.5
print('RMSE', score_LR)
LR_time = time.time() - start
print('Time:', LR_time)

RMSE 2677.0399208141225
Time: 19.464774131774902
CPU times: user 14.7 s, sys: 4.81 s, total: 19.5 s
Wall time: 19.5 s


In [26]:
model_RFR1 = RandomForestRegressor(max_depth = 10)
model_RFR2 = RandomForestRegressor(max_depth = 20, max_features = 'sqrt')
model_RFR3 = RandomForestRegressor(n_estimators = 200, max_depth = 20)

---

In [27]:
def rmse(target, predictions):
    return RMSE(target, predictions) ** 0.5

In [28]:
scorer = make_scorer(rmse, greater_is_better = False)

In [29]:
scores1 = cross_val_score(model_RFR1, features_train, target_train, cv = 5, scoring = scorer)
scores2 = cross_val_score(model_RFR2, features_train, target_train, cv = 5, scoring = scorer)
scores3 = cross_val_score(model_RFR3, features_train, target_train, cv = 5, scoring = scorer)

In [30]:
print('model_RFR1. RMSE:', abs(scores1).mean())
print('model_RFR2. RMSE:', abs(scores2).mean())
print('model_RFR3. RMSE:', abs(scores3).mean())

model_RFR1. RMSE: 1872.5997803171776
model_RFR2. RMSE: 2028.3182205661456
model_RFR3. RMSE: 1583.143022632804


In [41]:
model_RFR3.fit(features_train, target_train)
predictions = model_RFR3.predict(features_valid)
print('RMSE случайного леса:', RMSE(target_valid, predictions) ** 0.5)

RMSE случайного леса: 1560.7702446910175


In [32]:
%%time
start = time.time()
model_RFR = RandomForestRegressor()
model_RFR.fit(features_train, target_train)
predictions_RFR = model_RFR.predict(features_valid)
score_RFR = RMSE(target_valid, predictions_RFR) ** 0.5
print('RMSE:', score_RFR)
end = time.time()
RFR_time = end - start
print('Time:', RFR_time)

RMSE: 1617.285455117413
Time: 55.75129675865173
CPU times: user 55.2 s, sys: 364 ms, total: 55.5 s
Wall time: 55.8 s


##### Воспользуемся градиентным бустингом и библиотекой LightGBM

In [36]:
lgb_train = lgb.Dataset(category_features_train, category_target_train)
lgb_eval = lgb.Dataset(category_features_test, category_target_test, reference = lgb_train)

In [37]:
params = {
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': {'l2', 'l1'},
    'num_leaves': 31,
}

In [38]:
%%time
start = time.time()
gbm = lgb.train(params, lgb_train, valid_sets = lgb_eval)
LGB_time = time.time() - start

[1]	valid_0's l1: 3389.14	valid_0's l2: 1.8524e+07
[2]	valid_0's l1: 3131.43	valid_0's l2: 1.60078e+07
[3]	valid_0's l1: 2902.51	valid_0's l2: 1.39464e+07
[4]	valid_0's l1: 2697.68	valid_0's l2: 1.22335e+07
[5]	valid_0's l1: 2517.93	valid_0's l2: 1.08232e+07
[6]	valid_0's l1: 2360.42	valid_0's l2: 9.66885e+06
[7]	valid_0's l1: 2219.95	valid_0's l2: 8.70365e+06
[8]	valid_0's l1: 2094.13	valid_0's l2: 7.88819e+06
[9]	valid_0's l1: 1981.83	valid_0's l2: 7.19554e+06
[10]	valid_0's l1: 1882.6	valid_0's l2: 6.61593e+06
[11]	valid_0's l1: 1793.63	valid_0's l2: 6.13462e+06
[12]	valid_0's l1: 1714.3	valid_0's l2: 5.71751e+06
[13]	valid_0's l1: 1643.82	valid_0's l2: 5.36378e+06
[14]	valid_0's l1: 1581.47	valid_0's l2: 5.06313e+06
[15]	valid_0's l1: 1525.33	valid_0's l2: 4.79985e+06
[16]	valid_0's l1: 1476.67	valid_0's l2: 4.58262e+06
[17]	valid_0's l1: 1434.1	valid_0's l2: 4.39498e+06
[18]	valid_0's l1: 1396.49	valid_0's l2: 4.23736e+06
[19]	valid_0's l1: 1363.07	valid_0's l2: 4.09764e+06
[20]	v

In [39]:
%%time
start = time.time()
predictions_LGB = gbm.predict(category_features_test, num_iteration = gbm.best_iteration)
LGB2_time = time.time() - start

CPU times: user 1.34 s, sys: 4.04 ms, total: 1.35 s
Wall time: 1.3 s


In [40]:
print('RMSE:', RMSE(category_target_test, predictions_LGB) ** 0.5)
print('Time:', LGB_time + LGB2_time)

RMSE: 1678.331131445649
Time: 18.99661159515381


# 3. Анализ моделей

##### LightGBM показала себя очень неплохо, с временем работы чуть больше минуты, а итоговый RMSE - 1643

<div class="alert alert-block alert-warning">
<b>Модель случайного леса так же показала хорошие результаты, даже без подгонки параметров</b> 
</div>